In [ ]:
! pip install tqdm boto3 requests regex sentencepiece sacremoses
! pip install transformers
! pip install sentence_transformers
! pip install -U sentence-transformers
# ! pip install numpy
! pip install torch
! pip install torchtext
! pip install torchmetrics
! pip install pytorch-lightning
! pip install time
! pip install ipykernel
! pip install spacy
! pip install "grpcio>=1.37.0,<2.0" "h5py>=3.6.0,<3.7" "numpy>=1.22.3,<1.23.0"
# ! pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

import json
import pandas as pd
# Basics: dataset, data loaders, Classifier
import collections
import json
import torch
import torch.nn as nn
import numpy as np
import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
# from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import random
import re
# import preprocessor as p
from sklearn.cluster import KMeans # https://www.youtube.com/watch?v=Kc9gN_gODvQ
import matplotlib.pyplot as plt
import math
from google.colab import drive
drive.mount('/content/drive')





In [ ]:
import json
import pandas as pd
# Basics: dataset, data loaders, Classifier
import collections
import json
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
# from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import random
import re
# import preprocessor as p
from sklearn.cluster import KMeans # https://www.youtube.com/watch?v=Kc9gN_gODvQ
import matplotlib.pyplot as plt
import math
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class BERT_Data:
    '''
    '''
    def __init__(self, random_seed = 5,
                bert_model = 'bert-base-uncased', #'bert-base-uncased' or 'bert-large-uncased'
                date_range_begin = None, date_range_end = '2018-04-01',
                bills_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/bills_data/115th.csv',
                clean_bills_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/bills_data/115th_clean.csv',
                minimal_clean_bills_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/bills_data/115th_clean_minimal.csv',
                foxnews_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/fox.csv',
                clean_foxnews_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/fox_clean.csv',
                minimal_clean_foxnews_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/fox_clean_minimal.csv',
                breitbart_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/breitbart.csv',
                clean_breitbart_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/breitbart_clean.csv',
                minimal_clean_breitbart_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/breitbart_clean_minimal.csv',
                cnn_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/cnn.csv',
                clean_cnn_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/cnn_clean.csv',
                minimal_clean_cnn_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/cnn_clean_minimal.csv',
                nytimes_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/nyt.csv',
                clean_nytimes_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/nyt_clean.csv',
                minimal_clean_nytimes_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/nyt_clean_minimal.csv',
                truncated_minimal_clean_nytimes_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/nyt_clean_minimal_truncated.csv',
                wapo_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/washington_post_with_date.csv',
                clean_wapo_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/washington_post_with_date_clean.csv',
                minimal_clean_wapo_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/washington_post_with_date_clean_minimal.csv',
                truncated_minimal_clean_wapo_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/washington_post_with_date_clean_minimal_truncated.csv',
                master_key_csvpath = '/content/drive/Shareddrives/PulseofPolicy_data/news_data/v2_master_index_df.csv'
    ):
        self.device = self.cuda_mps_cpu()
        self.random_seed = random_seed
        self.random_seed_function()
        self.date_range_begin = date_range_begin
        self.date_range_end = date_range_end
        self.bills_csvpath = bills_csvpath
        self.clean_bills_csvpath = clean_bills_csvpath
        self.minimal_clean_bills_csvpath = minimal_clean_bills_csvpath
        self.df_bills_prepared = pd.read_csv(minimal_clean_bills_csvpath)
        # self.df_bills_raw = pd.read_csv(bills_csvpath)
        # self.df_bills_clean = self.clean_bills()
        # self.df_bills_clean = pd.read_csv(clean_bills_csvpath)
        self.foxnews_csvpath = foxnews_csvpath
        self.minimal_clean_foxnews_csvpath = minimal_clean_foxnews_csvpath
        self.breitbart_csvpath = breitbart_csvpath
        self.minimal_clean_breitbart_csvpath = minimal_clean_breitbart_csvpath
        self.cnn_csvpath = cnn_csvpath
        self.minimal_clean_cnn_csvpath = minimal_clean_cnn_csvpath
        self.nytimes_csvpath = nytimes_csvpath
        self.minimal_clean_nytimes_csvpath = minimal_clean_nytimes_csvpath
        self.truncated_minimal_clean_nytimes_csvpath = truncated_minimal_clean_nytimes_csvpath
        self.wapo_csvpath = wapo_csvpath
        self.minimal_clean_wapo_csvpath = minimal_clean_wapo_csvpath
        self.truncated_minimal_clean_wapo_csvpath = truncated_minimal_clean_wapo_csvpath
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model)
        self.bert_base = AutoModel.from_pretrained(bert_model)
        self.practice_bill = self.df_bills_prepared.head(5).copy()
        self.master_key_csvpath = master_key_csvpath
        self.df_master_key = pd.read_csv(master_key_csvpath)


    def clean_bills(self, only_2017 = False, only_2018 = False,
                save = True
    ):
        '''

        '''
        df = pd.read_csv(self.bills_csvpath)
        df.loc[:, ['new_index']] = df.index
        df.loc[:, 'cleaned_text'] = df.loc[:, 'raw_text'].apply(
                self.clean_bill_text, args=()
        )
        df.loc[:, ['date']] = pd.to_datetime(
                    df.loc[:, 'introduced_date'], format='%Y-%m-%d'
        )
        df.loc[:, ['house_passage_binary']] = df.loc[:, 'house_passage'].fillna(0, inplace=True)

        df.loc[:, ['house_passage_binary']] = np.where(
                df.loc[:, 'house_passage_binary'] != 0, 1, 0
        )

        if only_2017:
            df = df.loc[(df.loc[:, 'date'] >= '2017-01-01'
                    & df.loc[:, 'date'] < '2018-01-01'
            ), :]

        if only_2018:
            df = df.loc[(df.loc[:, 'date'] >= '2018-01-01'
                    & df.loc[:, 'date'] < '2019-01-01'
            ), :]

        df.loc[:, ['cleaned_text']] = df.loc[:, 'cleaned_text'].apply(
                self.clean_generalnews_text, args=()
        )
        df = df.loc[:, ['bill_id', 'new_index', 'cleaned_text', 'date', 'house_passage_binary', 'bill_type']]

        if save:
            df.to_csv(self.clean_bills_csvpath, index=False)

        return df


    def dates_clean_news(self, df_whole, Date_version, long_date_version,
            start_date = None, end_date = None,
            minimal_columns = False # minimal_columns = ['index', 'date', 'cleaned_text']
    ):
        '''

        '''
        if start_date is None:
            start_date = self.date_range_begin
        if end_date is None:
            end_date = self.date_range_end

        df = df_whole.copy()
        if Date_version:
            df.loc[:, ['date']] = pd.to_datetime(
                    df.loc[:, 'Date'], format='%Y-%m-%d'
            )
        elif long_date_version:
            df.loc[:, ['date']] = pd.to_datetime(
                    df.loc[:, 'date'].str[:10], format='%Y-%m-%d'
            )
        else:
            df.loc[:, ['date']] = pd.to_datetime(
                    df.loc[:, 'date'], format='%Y-%m-%d'
            )

        if start_date is not None:
            df = df.loc[df.loc[:, 'date'] >= start_date, :]
        if end_date is not None:
            df = df.loc[df.loc[:, 'date'] <= end_date, :]

        if minimal_columns is not False:
            df = df.loc[:, minimal_columns]

        return df


    def clean_foxnews(self, save = True):
        '''

        '''
        df_fox = pd.read_csv(self.foxnews_csvpath)
        df_fox.loc[:, ['new_index']] = df_fox.index
        df_fox_dated = self.dates_clean_news(df_fox, Date_version = True,
                long_date_version = False
        )
        df_fox_dated.loc[:, 'cleaned_text'] = df_fox_dated.loc[:, 'article_text'].apply(
                self.clean_foxnews_text, args=()
        )
        df_fox_dated = df_fox_dated.loc[:, ['uuid', 'new_index', 'cleaned_text', 'date']]

        if save:
            df_fox_dated.to_csv(self.minimal_clean_foxnews_csvpath, index=False)

        return df_fox_dated


    def clean_breitbart(self, save = True):
        '''

        '''
        df_breitbart = pd.read_csv(self.breitbart_csvpath)
        df_breitbart.loc[:, ['new_index']] = df_breitbart.index
        df_breitbart_dated = self.dates_clean_news(df_breitbart,
                Date_version = True, long_date_version = False
        )
        df_breitbart_dated.loc[:, 'cleaned_text'] = df_breitbart_dated.loc[:, 'article_text'].apply(
                self.clean_generalnews_text, args=()
        )
        df_breitbart_dated = df_breitbart_dated.loc[:, ['uuid', 'new_index', 'cleaned_text', 'date']]

        if save:
            df_breitbart_dated.to_csv(self.minimal_clean_breitbart_csvpath,
                    index=False
            )

        return df_breitbart_dated


    def clean_cnn(self, save = True):
        '''

        '''
        df_cnn = pd.read_csv(self.cnn_csvpath)
        df_cnn.loc[:, ['new_index']] = df_cnn.index
        df_cnn_dated = self.dates_clean_news(df_cnn, Date_version = False,
                long_date_version = True
        )
        df_cnn_dated.loc[:, 'cleaned_text'] = df_cnn_dated.loc[:, 'text'].apply(
                self.clean_generalnews_text, args=()
        )
        df_cnn_dated = df_cnn_dated.loc[:, ['title', 'new_index', 'cleaned_text', 'date']]

        if save:
            df_cnn_dated.to_csv(self.minimal_clean_cnn_csvpath, index=False)

        return df_cnn_dated


    def clean_nytimes(self, save = True):
        '''

        '''
        df_nytimes = pd.read_csv(self.nytimes_csvpath)
        df_nytimes.loc[:, ['new_index']] = df_nytimes.index
        df_nytimes_dated = self.dates_clean_news(df_nytimes,
                Date_version = False, long_date_version = True
        )
        df_nytimes_dated.loc[:, 'cleaned_text'] = df_nytimes_dated.loc[:, 'text'].apply(
                self.clean_generalnews_text, args=()
        )
        df_nytimes_dated = df_nytimes_dated.loc[:, ['title', 'new_index', 'cleaned_text', 'date']]

        if save:
            df_nytimes_dated.to_csv(self.minimal_clean_nytimes_csvpath,
                    index=False
            )

        return df_nytimes_dated


    def clean_wapo(self, save = True):
        '''

        '''
        df_wapo = pd.read_csv(self.wapo_csvpath)
        df_wapo.loc[:, ['new_index']] = df_wapo.index
        df_wapo_dated = self.dates_clean_news(df_wapo, Date_version = False,
                long_date_version = False
        )
        df_wapo_dated.loc[:, 'cleaned_text'] = df_wapo_dated.loc[:, 'text'].apply(
                self.clean_generalnews_text, args=()
        )
        df_wapo_dated = df_wapo_dated.loc[:, ['title', 'new_index', 'cleaned_text', 'date']]

        if save:
            df_wapo_dated.to_csv(self.minimal_clean_wapo_csvpath, index=False)

        return df_wapo_dated


    def clean_bill_text(self, text, glove=None, cbows=None, lemma=None):
        '''

        '''
        text = text.lower()
        text = re.compile('<.*?>').sub('', text)
        text = re.compile(r"[_]").sub(" ", text)
        text = re.sub(r"\[.*?\]", "", text)
        text = re.sub(r"\d+\sU\.S\.C\.\s\d+[a-z]?(\(\d+\))?( \([a-z]+\))?", "",
                text
        )
        text = re.sub(r'(sec\.\s+\d+\.?)|(section\s+\d+\.)', '', text)
        text = re.sub(r'\(\d+ u\.s\.c\. \d+\([a-z]\)(\(\d+\))*\)', '', text)
        text = re.sub(r'\(\d+\s+u\.s\.c\.\s+\d+\)', '', text)
        text = re.sub(r'\(\d+ u\.s\.c\. \d+\)', '', text)
        text = re.sub(r'\(\d+\)', '', text)
        text = re.sub(r'\([ivxlcdm]+\)', '', text)
        text = re.sub(r'\(\w\)', '', text)
        text = re.sub(r'\[\d+\]', '', text)
        text = re.sub(r'\n', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'([a-z])\.--([a-z])', r'\1. -- \2', text)
        text = text.replace('``', '"').replace("''", '"')
        text = text.strip()

        return text


    def clean_foxnews_text(self, text):
        '''
        Inputs:
            text: A uncleaned string of text
        Returns:
            text: A cleaned string of text
        '''
        text = text.replace("Get the latest updates from the 2024 campaign trail, exclusive interviews and more Fox News politics content.SubscribedYou've successfully subscribed to this newsletter!", " ")
        text = text.lower()
        text = re.sub(r"([.!?;:\"“”])(?=[^\s])|”(?=\w)", r"\1 ", text)
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()

        return text


    def clean_generalnews_text(self, text):
        '''

        '''
        text = re.sub(r"http\S+", " ", text)
        text = re.sub(r"@\S+", " ", text)
        text = re.sub(r'([a-z])\.”([A-Z])', r'\1. ” \2', text)
        text = text.lower()
        text = re.sub(r"([.!?;:\"“”])(?=[^\s])|”(?=\w)", r"\1 ", text)
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()

        return text


    def long_texts_dim_reduction(self, embedding_tensor,
                dim_reduction_strategy
    ): # Pooling, max, (potential: PCA)
        '''

        '''
        if dim_reduction_strategy == 'mean':
            embedding_tensor = embedding_tensor.mean(dim=0)
        elif dim_reduction_strategy == 'max':
            embedding_tensor = embedding_tensor.max(dim=0).values

        print(embedding_tensor.shape)
        return embedding_tensor


    def bert_embed_single_row(self, text, index = None, pooling = 'mean',
            max_len = 510, attention_mask_onpadding = True,
            long_dim_reduction_strategy = 'max',
            skip_long_texts = False, truncate_text = False,
    ): #512 tokens BERT # max, mean, CLS token embedding
        '''

        '''
        print(f'Bill/News index: {index}')
        if truncate_text:
            text_tokenized = self.tokenizer(text, return_tensors='pt',
                add_special_tokens=False, truncation=True, max_length = max_len
            )
        else:
            text_tokenized = self.tokenizer(text, return_tensors='pt',
                add_special_tokens=False, truncation=False
            )
        text_len = len(text_tokenized.input_ids[0])
        if skip_long_texts:
            if text_len > max_len:
                return None
        # print(text_tokenized.input_ids[0])
        text_segment_count = math.ceil(text_len / max_len)
        embedding_tensor = torch.tensor([], dtype = torch.float32)
        for segment in range(text_segment_count):
            start_token = segment * max_len
            end_token = min(start_token + max_len, text_len)
            # print(f'Segment {segment}, start_token {start_token}, end_token {end_token}')
            # print(self.tokenizer.cls_token_id, self.tokenizer.sep_token_id)
            segment_ids_tensor = text_tokenized.input_ids[0, start_token:end_token]
            padded_token_count = (max_len) - (end_token - start_token)
            if padded_token_count == 0:
                segment_ids_tensor = torch.cat([
                        torch.tensor([self.tokenizer.cls_token_id]),
                        segment_ids_tensor,
                        torch.tensor([self.tokenizer.sep_token_id])
                ]).unsqueeze(0)
            else:
                padding_tokens_tensor = torch.tensor(
                        [self.tokenizer.pad_token_id] * padded_token_count
                )
                segment_ids_tensor = torch.cat([
                        torch.tensor([self.tokenizer.cls_token_id]),
                        segment_ids_tensor,
                        padding_tokens_tensor,
                        torch.tensor([self.tokenizer.sep_token_id])
                ]).unsqueeze(0)
            if attention_mask_onpadding:
                attention_mask = (segment_ids_tensor !=
                        self.tokenizer.pad_token_id).to(dtype=torch.int64
                )
                with torch.no_grad():
                    segment_embeddings_tensor = self.bert_base(
                            input_ids = segment_ids_tensor,
                            attention_mask=attention_mask
                    ).last_hidden_state
            else:
                with torch.no_grad():
                    segment_embeddings_tensor = self.bert_base(
                            segment_ids_tensor
                    ).last_hidden_state

            if segment == 0:
                total_embeddings_tensor = segment_embeddings_tensor
            else:
                # print('total_embeddings_tensor: ', total_embeddings_tensor.shape)
                # print('segment_embeddings_tensor: ', segment_embeddings_tensor.shape)
                total_embeddings_tensor = torch.cat([total_embeddings_tensor,
                        segment_embeddings_tensor
                ])

        if pooling == 'mean':
            output_embedding_tensor = total_embeddings_tensor.mean(dim=1)
        elif pooling == 'max':
            output_embedding_tensor = total_embeddings_tensor.max(dim=1).values
        else: # None or CLS implies CLS
            output_embedding_tensor = total_embeddings_tensor[:, 0, :]

        if not skip_long_texts:
            if not truncate_text:
                output_embedding_tensor = self.long_texts_dim_reduction(
                        output_embedding_tensor, long_dim_reduction_strategy
                )

        print('output_embedding_tensor: ', output_embedding_tensor.shape)
        return output_embedding_tensor


    def total_bert_embeddings(self, df, text_column, pooling = 'mean',
            max_len = 510, attention_mask_onpadding = True,
            long_dim_reduction_strategy = 'max',
            skip_long_texts = False, truncate_text = False
    ):
        '''

        '''
        embeddings_lst = []

        for index, row in df.iterrows():
            ## Used in testing
            # if index == 10:
            #     break
            embedded_row =  self.bert_embed_single_row(row.loc[text_column],
                            index, pooling, max_len, attention_mask_onpadding,
                            long_dim_reduction_strategy, skip_long_texts,
                            truncate_text
            )
            # If skip_long_texts is True and text after tokenization longer than 512 than text skipped
            if embedded_row is not None:
                embeddings_lst.append(embedded_row)

        embeddings_tensor = torch.stack(embeddings_lst)

        return embeddings_tensor


    def cuda_mps_cpu(self):
        '''
        Determine if GPU or MPS or CPU is available
        '''
        if torch.cuda.is_available():  # use GPU if available
            print('Using GPU')
            return torch.device('cuda')
        # https://www.linkedin.com/pulse/how-use-gpu-tensorflow-pytorch-libraries-macbook-pro-m2apple-kashyap/
        elif torch.backends.mps.is_available():
            print('Using MPS')
            return torch.device('mps')
        else:
            print('Using CPU')
            return torch.device('cpu')


    def random_seed_function(self):
        '''
        Set random seed for reproducibility
        '''
        torch.manual_seed(self.random_seed)

        if self.device == 'cuda':
            torch.cuda.manual_seed_all(self.random_seed)

        random.seed(self.random_seed)

        return None

In [ ]:
class TorchEmbeddings:
    '''
    '''
    def __init__(self, random_seed = 5,
            bills_mean_mean = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/bills_embeddings/115th_embeddings_mean_mean.pt'),
            bills_cls = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/bills_embeddings/115th_embeddings_cls.pt'),
            bills_mean_max = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/bills_embeddings/115th_embeddings_mean_max.pt'),
            foxnews_mean_mean = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/foxnews_embeddings_mean_mean.pt'),
            foxnews_cls = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/foxnews_embeddings_cls.pt'),
            foxnews_mean_max = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/foxnews_embeddings_mean_max.pt'),
            breitbart_mean_mean = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/breitbart_embeddings_mean_mean.pt'),
            breitbart_cls = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/breitbart_embeddings_cls.pt'),
            breitbart_mean_max = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/breitbart_embeddings_mean_max.pt'),
            cnn_mean_mean = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/cnn_embeddings_mean_mean.pt'),
            cnn_cls = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/cnn_embeddings_cls.pt'),
            cnn_mean_max = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/cnn_embeddings_mean_max.pt'),
            nytimes_mean_mean = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/truncated_nytimes_embeddings_mean_max.pt'),
            nytimes_cls = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/truncated_nytimes_embeddings_cls.pt'),
            nytimes_mean_max = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/truncated_nytimes_embeddings_mean_max.pt'),
            wapo_mean_mean = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/truncated_wapo_embeddings_mean_mean.pt'),
            wapo_cls = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/truncated_wapo_embeddings_cls.pt'),
            wapo_mean_max = torch.load('/content/drive/Shareddrives/PulseofPolicy_data/news_embeddings/truncated_wapo_embeddings_mean_max.pt'),
    ):
        self.device = self.cuda_mps_cpu()
        self.random_seed = random_seed
        self.random_seed_function()
        self.bills_mean_mean = bills_mean_mean
        self.bills_cls = bills_cls
        self.bills_mean_max = bills_mean_max
        self.foxnews_mean_mean = foxnews_mean_mean
        self.foxnews_cls = foxnews_cls
        self.foxnews_mean_max = foxnews_mean_max
        self.breitbart_mean_mean = breitbart_mean_mean
        self.breitbart_cls = breitbart_cls
        self.breitbart_mean_max = breitbart_mean_max
        self.cnn_mean_mean = cnn_mean_mean
        self.cnn_cls = cnn_cls
        self.cnn_mean_max = cnn_mean_max
        self.nytimes_mean_mean = nytimes_mean_mean
        self.nytimes_cls = nytimes_cls
        self.nytimes_mean_max = nytimes_mean_max
        self.wapo_mean_mean = wapo_mean_mean
        self.wapo_cls = wapo_cls
        self.wapo_mean_max = wapo_mean_max


    def cuda_mps_cpu(self):
        '''
        Determine if GPU or MPS or CPU is available
        '''
        if torch.cuda.is_available():  # use GPU if available
            print('Using GPU')
            return torch.device('cuda')
        # https://www.linkedin.com/pulse/how-use-gpu-tensorflow-pytorch-libraries-macbook-pro-m2apple-kashyap/
        elif torch.backends.mps.is_available():
            print('Using MPS')
            return torch.device('mps')
        else:
            print('Using CPU')
            return torch.device('cpu')


    def random_seed_function(self):
        '''
        Set random seed for reproducibility
        '''
        torch.manual_seed(self.random_seed)

        if self.device == 'cuda':
            torch.cuda.manual_seed_all(self.random_seed)

        random.seed(self.random_seed)

        return None

In [ ]:
bill_month_lst = ['bill_index_2017-01', 'bill_index_2017-02', 'bill_index_2017-03', 'bill_index_2017-04', 'bill_index_2017-05', 'bill_index_2017-06', 'bill_index_2017-07', 'bill_index_2017-08', 'bill_index_2017-09', 'bill_index_2017-10']

foxnews_month_lst = [('foxnews_index_2016-11', 'foxnews_index_2016-12', 'foxnews_index_2017-01', 'foxnews_index_2017-02', 'foxnews_index_2017-03'), (
        'foxnews_index_2016-12', 'foxnews_index_2017-01', 'foxnews_index_2017-02', 'foxnews_index_2017-03', 'foxnews_index_2017-04'), (
        'foxnews_index_2017-01', 'foxnews_index_2017-02', 'foxnews_index_2017-03', 'foxnews_index_2017-04', 'foxnews_index_2017-05'), (
        'foxnews_index_2017-02', 'foxnews_index_2017-03', 'foxnews_index_2017-04', 'foxnews_index_2017-05', 'foxnews_index_2017-06'), (
        'foxnews_index_2017-03', 'foxnews_index_2017-04', 'foxnews_index_2017-05', 'foxnews_index_2017-06', 'foxnews_index_2017-07'), (
        'foxnews_index_2017-04', 'foxnews_index_2017-05', 'foxnews_index_2017-06', 'foxnews_index_2017-07', 'foxnews_index_2017-08'), (
        'foxnews_index_2017-05', 'foxnews_index_2017-06', 'foxnews_index_2017-07', 'foxnews_index_2017-08', 'foxnews_index_2017-09'), (
        'foxnews_index_2017-06', 'foxnews_index_2017-07', 'foxnews_index_2017-08', 'foxnews_index_2017-09', 'foxnews_index_2017-10'), (
        'foxnews_index_2017-07', 'foxnews_index_2017-08', 'foxnews_index_2017-09', 'foxnews_index_2017-10', 'foxnews_index_2017-11'), (
        'foxnews_index_2017-08', 'foxnews_index_2017-09', 'foxnews_index_2017-10', 'foxnews_index_2017-11', 'foxnews_index_2017-12')]

breitbart_month_lst = [('breitbart_index_2016-11', 'breitbart_index_2016-12', 'breitbart_index_2017-01', 'breitbart_index_2017-02', 'breitbart_index_2017-03'), (
        'breitbart_index_2016-12', 'breitbart_index_2017-01', 'breitbart_index_2017-02', 'breitbart_index_2017-03', 'breitbart_index_2017-04'), (
        'breitbart_index_2017-01', 'breitbart_index_2017-02', 'breitbart_index_2017-03', 'breitbart_index_2017-04', 'breitbart_index_2017-05'), (
        'breitbart_index_2017-02', 'breitbart_index_2017-03', 'breitbart_index_2017-04', 'breitbart_index_2017-05', 'breitbart_index_2017-06'), (
        'breitbart_index_2017-03', 'breitbart_index_2017-04', 'breitbart_index_2017-05', 'breitbart_index_2017-06', 'breitbart_index_2017-07'), (
        'breitbart_index_2017-04', 'breitbart_index_2017-05', 'breitbart_index_2017-06', 'breitbart_index_2017-07', 'breitbart_index_2017-08'), (
        'breitbart_index_2017-05', 'breitbart_index_2017-06', 'breitbart_index_2017-07', 'breitbart_index_2017-08', 'breitbart_index_2017-09'), (
        'breitbart_index_2017-06', 'breitbart_index_2017-07', 'breitbart_index_2017-08', 'breitbart_index_2017-09', 'breitbart_index_2017-10'), (
        'breitbart_index_2017-07', 'breitbart_index_2017-08', 'breitbart_index_2017-09', 'breitbart_index_2017-10', 'breitbart_index_2017-11'), (
        'breitbart_index_2017-08', 'breitbart_index_2017-09', 'breitbart_index_2017-10', 'breitbart_index_2017-11', 'breitbart_index_2017-12')]

cnn_month_lst = [('cnn_index_2016-11', 'cnn_index_2016-12', 'cnn_index_2017-01', 'cnn_index_2017-02', 'cnn_index_2017-03'), (
        'cnn_index_2016-12', 'cnn_index_2017-01', 'cnn_index_2017-02', 'cnn_index_2017-03', 'cnn_index_2017-04'), (
        'cnn_index_2017-01', 'cnn_index_2017-02', 'cnn_index_2017-03', 'cnn_index_2017-04', 'cnn_index_2017-05'), (
        'cnn_index_2017-02', 'cnn_index_2017-03', 'cnn_index_2017-04', 'cnn_index_2017-05', 'cnn_index_2017-06'), (
        'cnn_index_2017-03', 'cnn_index_2017-04', 'cnn_index_2017-05', 'cnn_index_2017-06', 'cnn_index_2017-07'), (
        'cnn_index_2017-04', 'cnn_index_2017-05', 'cnn_index_2017-06', 'cnn_index_2017-07', 'cnn_index_2017-08'), (
        'cnn_index_2017-05', 'cnn_index_2017-06', 'cnn_index_2017-07', 'cnn_index_2017-08', 'cnn_index_2017-09'), (
        'cnn_index_2017-06', 'cnn_index_2017-07', 'cnn_index_2017-08', 'cnn_index_2017-09', 'cnn_index_2017-10'), (
        'cnn_index_2017-07', 'cnn_index_2017-08', 'cnn_index_2017-09', 'cnn_index_2017-10', 'cnn_index_2017-11'), (
        'cnn_index_2017-08', 'cnn_index_2017-09', 'cnn_index_2017-10', 'cnn_index_2017-11', 'cnn_index_2017-12')]

nytimes_month_lst = [('nytimes_index_2016-11', 'nytimes_index_2016-12', 'nytimes_index_2017-01', 'nytimes_index_2017-02', 'nytimes_index_2017-03'), (
        'nytimes_index_2016-12', 'nytimes_index_2017-01', 'nytimes_index_2017-02', 'nytimes_index_2017-03', 'nytimes_index_2017-04'), (
        'nytimes_index_2017-01', 'nytimes_index_2017-02', 'nytimes_index_2017-03', 'nytimes_index_2017-04', 'nytimes_index_2017-05'), (
        'nytimes_index_2017-02', 'nytimes_index_2017-03', 'nytimes_index_2017-04', 'nytimes_index_2017-05', 'nytimes_index_2017-06'), (
        'nytimes_index_2017-03', 'nytimes_index_2017-04', 'nytimes_index_2017-05', 'nytimes_index_2017-06', 'nytimes_index_2017-07'), (
        'nytimes_index_2017-04', 'nytimes_index_2017-05', 'nytimes_index_2017-06', 'nytimes_index_2017-07', 'nytimes_index_2017-08'), (
        'nytimes_index_2017-05', 'nytimes_index_2017-06', 'nytimes_index_2017-07', 'nytimes_index_2017-08', 'nytimes_index_2017-09'), (
        'nytimes_index_2017-06', 'nytimes_index_2017-07', 'nytimes_index_2017-08', 'nytimes_index_2017-09', 'nytimes_index_2017-10'), (
        'nytimes_index_2017-07', 'nytimes_index_2017-08', 'nytimes_index_2017-09', 'nytimes_index_2017-10', 'nytimes_index_2017-11'), (
        'nytimes_index_2017-08', 'nytimes_index_2017-09', 'nytimes_index_2017-10', 'nytimes_index_2017-11', 'nytimes_index_2017-12')]

wapo_month_lst = [('wapo_index_2016-11', 'wapo_index_2016-12', 'wapo_index_2017-01', 'wapo_index_2017-02', 'wapo_index_2017-03'), (
        'wapo_index_2016-12', 'wapo_index_2017-01', 'wapo_index_2017-02', 'wapo_index_2017-03', 'wapo_index_2017-04'), (
        'wapo_index_2017-01', 'wapo_index_2017-02', 'wapo_index_2017-03', 'wapo_index_2017-04', 'wapo_index_2017-05'), (
        'wapo_index_2017-02', 'wapo_index_2017-03', 'wapo_index_2017-04', 'wapo_index_2017-05', 'wapo_index_2017-06'), (
        'wapo_index_2017-03', 'wapo_index_2017-04', 'wapo_index_2017-05', 'wapo_index_2017-06', 'wapo_index_2017-07'), (
        'wapo_index_2017-04', 'wapo_index_2017-05', 'wapo_index_2017-06', 'wapo_index_2017-07', 'wapo_index_2017-08'), (
        'wapo_index_2017-05', 'wapo_index_2017-06', 'wapo_index_2017-07', 'wapo_index_2017-08', 'wapo_index_2017-09'), (
        'wapo_index_2017-06', 'wapo_index_2017-07', 'wapo_index_2017-08', 'wapo_index_2017-09', 'wapo_index_2017-10'), (
        'wapo_index_2017-07', 'wapo_index_2017-08', 'wapo_index_2017-09', 'wapo_index_2017-10', 'wapo_index_2017-11'), (
        'wapo_index_2017-08', 'wapo_index_2017-09', 'wapo_index_2017-10', 'wapo_index_2017-11', 'wapo_index_2017-12')]

In [ ]:
def min_max_norm(torch_tensor):
    '''
    Normalize a tensor to between 0 and 1 using min-max normalization
    '''
    normalized_tensor = (torch_tensor - torch_tensor.min()) / (torch_tensor.max() - torch_tensor.min())

    return normalized_tensor


def z_score_norm(torch_tensor):
    '''
    Normalize a tensor to between 0 and 1 using z-score normalization
    '''
    normalized_tensor = (torch_tensor - torch_tensor.mean()) / torch_tensor.std()

    return normalized_tensor



In [ ]:
def cosine_similarity(bills_tensor, news_tensor):
    '''
    Calculate cosine similarity between two tensors
    '''
    return F.cosine_similarity(bills_tensor, news_tensor)


In [ ]:
data_class = BERT_Data()
embeddings_class = TorchEmbeddings()

Using GPU


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Using GPU


<ipython-input-4-a8b2169d19ce>:57: DtypeWarning: Columns (8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df_master_key = pd.read_csv(master_key_csvpath)


In [ ]:
embeddings_class.bills_mean_mean.shape

torch.Size([8876, 768])

In [ ]:
embeddings_class.bills_mean_mean[a].shape

torch.Size([3, 768])

In [ ]:
master_key_df =  data_class.df_master_key

In [ ]:
#################################################
#      FoxNews Mean Mean Cosine Similarity      #
#################################################
bill_count = 0
cosine_sim = []
bill_index_lst = []
df = master_key_df.copy()
for index, month in enumerate(bill_month_lst):
    # bill_df = master_key_df.copy()
    # bill_indices = torch.tensor(bill_df.dropna(subset=[month]).loc[:, 'new_index'].tolist())
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    foxnews_month_tuple = foxnews_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_mean[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        print(f'bill_count: {bill_count}')
        bill_count += 1
        for foxnews_month in foxnews_month_tuple:
            # fox_df = master_key_df.copy()
            # fox_embed_indices = torch.tensor(fox_df.dropna(subset=[foxnews_month]).loc[:, 'new_index'].tolist())
            fox_indices = torch.tensor(master_key_df.dropna(subset=[foxnews_month])['new_index'].tolist())
            fox_normalized = embeddings_class.foxnews_mean_mean[fox_indices]
            bill_fox_cosine_sim = cosine_similarity(bill_normalized, fox_normalized)
            # print(f'________bill_fox_cosine_sim___________{bill_fox_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_fox_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)

df_cs_meanmean = pd.DataFrame(cosine_sim)

df_cs_meanmean.loc[:, 'bill_index'] = bill_index_lst


print(len(cosine_sim))

bill_count: 0
bill_count: 1
bill_count: 2
bill_count: 3
bill_count: 4
bill_count: 5
bill_count: 6
bill_count: 7
bill_count: 8
bill_count: 9
bill_count: 10
bill_count: 11
bill_count: 12
bill_count: 13
bill_count: 14
bill_count: 15
bill_count: 16
bill_count: 17
bill_count: 18
bill_count: 19
bill_count: 20
bill_count: 21
bill_count: 22
bill_count: 23
bill_count: 24
bill_count: 25
bill_count: 26
bill_count: 27
bill_count: 28
bill_count: 29
bill_count: 30
bill_count: 31
bill_count: 32
bill_count: 33
bill_count: 34
bill_count: 35
bill_count: 36
bill_count: 37
bill_count: 38
bill_count: 39
bill_count: 40
bill_count: 41
bill_count: 42
bill_count: 43
bill_count: 44
bill_count: 45
bill_count: 46
bill_count: 47
bill_count: 48
bill_count: 49
bill_count: 50
bill_count: 51
bill_count: 52
bill_count: 53
bill_count: 54
bill_count: 55
bill_count: 56
bill_count: 57
bill_count: 58
bill_count: 59
bill_count: 60
bill_count: 61
bill_count: 62
bill_count: 63
bill_count: 64
bill_count: 65
bill_count: 66
bill_

In [ ]:
df_cs_meanmean.loc[:, 'new_index'] = df_cs_meanmean.loc[:, 'bill_index']

In [ ]:
df_cs_meanmean.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_fox_meanmean.csv')

In [ ]:
df_cs_meanmean.head()

,0,1,2,3,4,5,6,7,8,9,...,1462,1463,1464,1465,1466,1467,1468,1469,1470,bill_index
0,0.995957,0.996310,0.996479,0.996278,0.995709,0.996115,0.995422,0.996451,0.996410,0.996259,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7966
1,0.998065,0.998144,0.998138,0.998105,0.997981,0.998188,0.997890,0.998270,0.998167,0.998109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7967
2,0.999259,0.999033,0.999256,0.999100,0.999208,0.999189,0.999261,0.999177,0.999286,0.999197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7968
3,0.998356,0.998457,0.998603,0.998441,0.998233,0.998419,0.998098,0.998571,0.998609,0.998492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7969
4,0.997576,0.997707,0.997737,0.997667,0.997408,0.997681,0.997227,0.997858,0.997759,0.997697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7970


In [ ]:
################################################
#      FoxNews Mean Max Cosine Similarity      #
################################################
bill_count = 0
cosine_sim = []
bill_index_lst = []
df = master_key_df.copy()
for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    foxnews_month_tuple = foxnews_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_max[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        print(f'bill_count: {bill_count}')
        bill_count += 1
        for foxnews_month in foxnews_month_tuple:
            fox_indices = torch.tensor(master_key_df.dropna(subset=[foxnews_month])['new_index'].tolist())
            fox_normalized = embeddings_class.foxnews_mean_max[fox_indices]
            bill_fox_cosine_sim = cosine_similarity(bill_normalized, fox_normalized)
            # print(f'________bill_fox_cosine_sim___________{bill_fox_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_fox_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)
    #     break
    # break


df_cs_meanmax = pd.DataFrame(cosine_sim)

df_cs_meanmax.loc[:, 'bill_index'] = bill_index_lst

df_cs_meanmax.loc[:, 'new_index'] = df_cs_meanmax.loc[:, 'bill_index']

df_cs_meanmax.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_fox_meanmax.csv')

# print(len(cosine_sim))
# print(cosine_sim)

bill_count: 0
bill_count: 1
bill_count: 2
bill_count: 3
bill_count: 4
bill_count: 5
bill_count: 6
bill_count: 7
bill_count: 8
bill_count: 9
bill_count: 10
bill_count: 11
bill_count: 12
bill_count: 13
bill_count: 14
bill_count: 15
bill_count: 16
bill_count: 17
bill_count: 18
bill_count: 19
bill_count: 20
bill_count: 21
bill_count: 22
bill_count: 23
bill_count: 24
bill_count: 25
bill_count: 26
bill_count: 27
bill_count: 28
bill_count: 29
bill_count: 30
bill_count: 31
bill_count: 32
bill_count: 33
bill_count: 34
bill_count: 35
bill_count: 36
bill_count: 37
bill_count: 38
bill_count: 39
bill_count: 40
bill_count: 41
bill_count: 42
bill_count: 43
bill_count: 44
bill_count: 45
bill_count: 46
bill_count: 47
bill_count: 48
bill_count: 49
bill_count: 50
bill_count: 51
bill_count: 52
bill_count: 53
bill_count: 54
bill_count: 55
bill_count: 56
bill_count: 57
bill_count: 58
bill_count: 59
bill_count: 60
bill_count: 61
bill_count: 62
bill_count: 63
bill_count: 64
bill_count: 65
bill_count: 66
bill_

In [ ]:
cosine_sim[0]

[0.7239527702331543,
 0.7448690533638,
 0.73344486951828,
 0.745381236076355,
 0.6951199769973755,
 0.7494260668754578,
 0.5446577072143555,
 0.8297791481018066,
 0.8095749616622925,
 0.7810891270637512,
 0.7573438882827759,
 0.7705519199371338,
 0.7685400247573853,
 0.7473514080047607,
 0.6888864040374756,
 0.7688370943069458,
 0.7239527702331543,
 0.6205537915229797,
 0.7443255186080933,
 0.6897242069244385,
 0.6801470518112183,
 0.7498696446418762,
 0.732654333114624,
 0.7299622297286987,
 0.7146973013877869,
 0.7855738401412964,
 0.750297486782074,
 0.7704339027404785,
 0.7499266862869263,
 0.7375101447105408,
 0.7819035053253174,
 0.7337679266929626,
 0.7322953343391418,
 0.745421826839447,
 0.6849954724311829,
 0.8264778852462769,
 0.7401852607727051,
 0.7210647463798523,
 0.6985242962837219,
 0.6984363794326782,
 0.7206304669380188,
 0.7877084612846375,
 0.7121902108192444,
 0.8078162670135498,
 0.8419044017791748,
 0.7828363180160522,
 0.7734863758087158,
 0.6984363794326782,
 

In [ ]:
bill_foxnews_cossim_array = np.array(cosine_sim)

np.save('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/bill_foxnews_cossim_array.npy', bill_foxnews_cossim_array)

<ipython-input-27-c359971a4bcd>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bill_foxnews_cossim_array = np.array(cosine_sim)


In [ ]:
###########################################
#      FoxNews CLS Cosine Similarity      #
###########################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    foxnews_month_tuple = foxnews_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_cls[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        print(f'bill_count: {bill_count}')
        bill_count += 1
        for foxnews_month in foxnews_month_tuple:
            fox_indices = torch.tensor(master_key_df.dropna(subset=[foxnews_month])['new_index'].tolist())
            fox_normalized = embeddings_class.foxnews_cls[fox_indices]
            bill_fox_cosine_sim = cosine_similarity(bill_normalized, fox_normalized)
            # print(f'________bill_fox_cosine_sim___________{bill_fox_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_fox_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)
    #     break
    # break


df_cs_cls = pd.DataFrame(cosine_sim)

df_cs_cls.loc[:, 'bill_index'] = bill_index_lst

df_cs_cls.loc[:, 'new_index'] = df_cs_cls.loc[:, 'bill_index']

df_cs_cls.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_fox_cls.csv')

# print(len(cosine_sim))
# print(cosine_sim)

In [ ]:
##################################################
#      Breitbart Mean Mean Cosine Similarity      #
##################################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    breitbart_month_tuple = breitbart_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_mean[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for breitbart_month in breitbart_month_tuple:
            breitbart_indices = torch.tensor(master_key_df.dropna(subset=[breitbart_month])['new_index'].tolist())
            breitbart_normalized = embeddings_class.breitbart_mean_mean[breitbart_indices]
            bill_breitbart_cosine_sim = cosine_similarity(bill_normalized, breitbart_normalized)
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_breitbart_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)


df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_breitbart_meanmean.csv')

print(len(cosine_sim))


4992


In [ ]:
##################################################
#      Breitbart Mean Max Cosine Similarity      #
##################################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    breitbart_month_tuple = breitbart_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_max[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for breitbart_month in breitbart_month_tuple:
            breitbart_indices = torch.tensor(master_key_df.dropna(subset=[breitbart_month])['new_index'].tolist())
            breitbart_normalized = embeddings_class.breitbart_mean_max[breitbart_indices]
            bill_breitbart_cosine_sim = cosine_similarity(bill_normalized, breitbart_normalized)
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_breitbart_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)


df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_breitbart_meanmax.csv')

print(len(cosine_sim))


4992


In [ ]:
#############################################
#      Breitbart CLS Cosine Similarity      #
#############################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    breitbart_month_tuple = breitbart_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_cls[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for breitbart_month in breitbart_month_tuple:
            breitbart_indices = torch.tensor(master_key_df.dropna(subset=[breitbart_month])['new_index'].tolist())
            breitbart_normalized = embeddings_class.breitbart_cls[breitbart_indices]
            bill_breitbart_cosine_sim = cosine_similarity(bill_normalized, breitbart_normalized)
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_breitbart_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)


df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_breitbart_cls.csv')

print(len(cosine_sim))


In [ ]:
bill_breitbart_cossim_array = np.array(cosine_sim)

np.save('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/bill_breitbart_cossim_array.npy', bill_breitbart_cossim_array)

<ipython-input-30-5caeb5c00b40>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bill_breitbart_cossim_array = np.array(cosine_sim)


In [ ]:
############################################
#      CNN Mean Mean Cosine Similarity      #
############################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    cnn_month_tuple = cnn_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_mean[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for cnn_month in cnn_month_tuple:
            cnn_indices = torch.tensor(master_key_df.dropna(subset=[cnn_month])['new_index'].tolist())
            cnn_normalized = embeddings_class.cnn_mean_mean[cnn_indices]
            bill_cnn_cosine_sim = bill_normalized, cnn_normalized
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_cnn_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)

df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_cnn_meanmean.csv')

print(len(cosine_sim))

In [ ]:
############################################
#      CNN Mean Max Cosine Similarity      #
############################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    cnn_month_tuple = cnn_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_max[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for cnn_month in cnn_month_tuple:
            cnn_indices = torch.tensor(master_key_df.dropna(subset=[cnn_month])['new_index'].tolist())
            cnn_normalized = embeddings_class.cnn_mean_max[cnn_indices]
            bill_cnn_cosine_sim = bill_normalized, cnn_normalized
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_cnn_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)

df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_cnn_meanmax.csv')

print(len(cosine_sim))

In [ ]:
#######################################
#      CNN CLS Cosine Similarity      #
#######################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    cnn_month_tuple = cnn_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_max[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for cnn_month in cnn_month_tuple:
            cnn_indices = torch.tensor(master_key_df.dropna(subset=[cnn_month])['new_index'].tolist())
            cnn_normalized = embeddings_class.cnn_mean_max[cnn_indices]
            bill_cnn_cosine_sim = bill_normalized, cnn_normalized
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_cnn_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)

df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_cnn_cls.csv')

print(len(cosine_sim))

In [ ]:
#################################################
#      NYTimes Mean Mean Cosine Similarity      #
#################################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    nytimes_month_tuple = nytimes_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_mean[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for nytimes_month in nytimes_month_tuple:
            nytimes_indices = torch.tensor(master_key_df.dropna(subset=[nytimes_month])['new_index'].tolist())
            nytimes_normalized = embeddings_class.nytimes_mean_mean[nytimes_indices]
            bill_nytimes_cosine_sim = cosine_similarity(bill_normalized, nytimes_normalized)
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_nytimes_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)

df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_nytimes_meanmean.csv')

print(len(cosine_sim))

In [ ]:
################################################
#      NYTimes Mean Max Cosine Similarity      #
################################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    nytimes_month_tuple = nytimes_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_max[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for nytimes_month in nytimes_month_tuple:
            nytimes_indices = torch.tensor(master_key_df.dropna(subset=[nytimes_month])['new_index'].tolist())
            nytimes_normalized = embeddings_class.nytimes_mean_max[nytimes_indices]
            bill_nytimes_cosine_sim = cosine_similarity(bill_normalized, nytimes_normalized)
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_nytimes_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)

df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_nytimes_meanmax.csv')

print(len(cosine_sim))

In [ ]:
###########################################
#      NYTimes CLS Cosine Similarity      #
###########################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    nytimes_month_tuple = nytimes_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_max[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for nytimes_month in nytimes_month_tuple:
            nytimes_indices = torch.tensor(master_key_df.dropna(subset=[nytimes_month])['new_index'].tolist())
            nytimes_normalized = embeddings_class.nytimes_mean_max[nytimes_indices]
            bill_nytimes_cosine_sim = cosine_similarity(bill_normalized, nytimes_normalized)
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_nytimes_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)

df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_nytimes_cls.csv')

print(len(cosine_sim))

In [ ]:
bill_nytimes_cossim_array = np.array(cosine_sim)

np.save('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/bill_cnn_cossim_array.npy', bill_nytimes_cossim_array)

<ipython-input-34-76e379bbbdac>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bill_nytimes_cossim_array = np.array(cosine_sim)


In [ ]:
##############################################
#      WaPo Mean Mean Cosine Similarity      #
##############################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    wapo_month_tuple = wapo_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_mean[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for wapo_month in wapo_month_tuple:
            wapo_indices = torch.tensor(master_key_df.dropna(subset=[wapo_month])['new_index'].tolist())
            wapo_normalized = embeddings_class.wapo_mean_mean[wapo_indices]
            bill_wapo_cosine_sim = cosine_similarity(bill_normalized, wapo_normalized)
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_wapo_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)

df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_wapo_meanmean.csv')

print(len(cosine_sim))


4992


In [ ]:
bill_wapo_cossim_array = np.array(cosine_sim)

np.save('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/wapo_cnn_cossim_array.npy', bill_wapo_cossim_array)

<ipython-input-39-c964a9659c33>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bill_wapo_cossim_array = np.array(cosine_sim)


In [ ]:
#############################################
#      WaPo Mean Max Cosine Similarity      #
#############################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    wapo_month_tuple = wapo_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_mean_max[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for wapo_month in wapo_month_tuple:
            wapo_indices = torch.tensor(master_key_df.dropna(subset=[wapo_month])['new_index'].tolist())
            wapo_normalized = embeddings_class.wapo_mean_max[wapo_indices]
            bill_wapo_cosine_sim = cosine_similarity(bill_normalized, wapo_normalized)
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_wapo_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)

df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_wapo_meanmax.csv')

print(len(cosine_sim))


4992


In [ ]:
########################################
#      WaPo CLS Cosine Similarity      #
########################################
bill_count = 0
cosine_sim = []
bill_index_lst = []

for index, month in enumerate(bill_month_lst):
    bill_indices = torch.tensor(master_key_df.dropna(subset=[month])['new_index'].tolist())
    # print('bill_indices_len:', len(bill_indices))
    bill_index_lst += bill_indices.tolist()
    wapo_month_tuple = wapo_month_lst[index]
    for bill_index in bill_indices:
        bill_normalized = embeddings_class.bills_cls[bill_index].unsqueeze(0)
        bill_cosine_sim_lst = []
        for wapo_month in wapo_month_tuple:
            wapo_indices = torch.tensor(master_key_df.dropna(subset=[wapo_month])['new_index'].tolist())
            wapo_normalized = embeddings_class.wapo_cls[wapo_indices]
            bill_wapo_cosine_sim = cosine_similarity(bill_normalized, wapo_normalized)
            # print(f'________bill_breitbart_cosine_sim___________{bill_breitbart_cosine_sim.shape}')
            bill_cosine_sim_lst += bill_wapo_cosine_sim.tolist()
        cosine_sim.append(bill_cosine_sim_lst)

df_cs = pd.DataFrame(cosine_sim)

df_cs.loc[:, 'bill_index'] = bill_index_lst

df_cs.loc[:, 'new_index'] = df_cs.loc[:, 'bill_index']

df_cs.to_csv('/content/drive/Shareddrives/PulseofPolicy_data/text_cosinesimilarity_scores/df_wapo_cls.csv')

print(len(cosine_sim))
